# Objective: Evaluate and compare the performance of several regression models for best performance in predicting life expectancy at birth.

# Procedure:

1.  Using GridSearchCV and Pipeline Tools to test and compare results of the following models:<br>
    a. LogisticRegression <br>
    b. Lasso <br>
    c. Ridge <br>
    d. Elastic Net
    
Which linear regression model is most accurate?


2.  Study regression model performance effect using FeatureReduction techniques: <br>
    a. SelectFromModel <br>
    b. Princial Component Analysis <br>
    c. Univariate Statistics

Is the best performing model from step 1 remain unchanged? <br> 

What do the FeatureReduction results say about the dataset and modeling performance?


# Importing Libraries

In [434]:
# pandas, numpy, matplotlib, seaborn, plotly libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns


%matplotlib inline

In [435]:
# sklearn libraries

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.decomposition import PCA

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet

from sklearn import linear_model


from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline



# Import Dataframe

## Select feature columns

### - Eleven features are selected from df_final, the dataframe containing all compiled data. 

1. 'State'
2. 'County Name'
3. 'e(0)'.  Life expectancy at birth.  This is the target.
4. 'Percent of adults with less than a high school diploma, 2015-19',
5. 'Percent of adults with a high school diploma only, 2015-19',
6. "Percent of adults completing some college or associate's degree, 2015-19",
7. "Percent of adults with a bachelor's degree or higher, 2015-19", 'PCTPOV017_2019',
8. 'Unemployment_rate_2020', 
9. 'Median_Household_Income_2019',
10. 'Med_HH_Income_Percent_of_State_Total_2019'
11. 'PCTPOV017_2019'


In [436]:
df = pd.read_csv('./df_final')

In [437]:
df.columns.values

array(['Unnamed: 0', 'Tract ID', 'STATE2KX_x', 'CNTY2KX', 'TRACT2KX',
       'e(0)', 'se(e(0))', 'Abridged life table flag', 'State',
       'County Name', 'STATE2KX_y', 'STUSAB', 'STATENS', 'FIPS_Code',
       'Area_name_x', 'Rural_urban_continuum_code_2013',
       'Urban_influence_code_2013', 'Metro_2013',
       'Civilian_labor_force_2000', 'Employed_2000', 'Unemployed_2000',
       'Unemployment_rate_2000', 'Civilian_labor_force_2001',
       'Employed_2001', 'Unemployed_2001', 'Unemployment_rate_2001',
       'Civilian_labor_force_2002', 'Employed_2002', 'Unemployed_2002',
       'Unemployment_rate_2002', 'Civilian_labor_force_2003',
       'Employed_2003', 'Unemployed_2003', 'Unemployment_rate_2003',
       'Civilian_labor_force_2004', 'Employed_2004', 'Unemployed_2004',
       'Unemployment_rate_2004', 'Civilian_labor_force_2005',
       'Employed_2005', 'Unemployed_2005', 'Unemployment_rate_2005',
       'Civilian_labor_force_2006', 'Employed_2006', 'Unemployed_2006',
       '

In [438]:
# Select features for use in regression modeling

df_ = df[['State', 'County Name', 'e(0)', 'Percent of adults with less than a high school diploma, 2015-19',
         'Percent of adults with a high school diploma only, 2015-19',
         "Percent of adults completing some college or associate's degree, 2015-19",
         "Percent of adults with a bachelor's degree or higher, 2015-19", 'PCTPOV017_2019',
         'Unemployment_rate_2020', 'Median_Household_Income_2019',
         'Med_HH_Income_Percent_of_State_Total_2019']]

print(df_.shape)
df_.sample(5)

(65662, 11)


,State,County Name,e(0),"Percent of adults with less than a high school diploma, 2015-19","Percent of adults with a high school diploma only, 2015-19","Percent of adults completing some college or associate's degree, 2015-19","Percent of adults with a bachelor's degree or higher, 2015-19",PCTPOV017_2019,Unemployment_rate_2020,Median_Household_Income_2019,Med_HH_Income_Percent_of_State_Total_2019
52200,Pennsylvania,Huntingdon,78.7,10.540301,48.825630,23.215540,17.418528,17.5,10.4,54323.0,85.608696
64116,Washington,King,80.8,6.887616,14.995393,25.623768,52.493221,7.9,7.5,102338.0,130.078552
28138,Maryland,Worcester,73.3,8.675672,31.805717,30.472757,29.045855,15.5,11.2,65821.0,75.967178
2551,Arkansas,Arkansas,73.1,16.091122,38.895596,29.244144,15.769138,24.1,4.0,52539.0,107.178703
17507,Georgia,Forsyth,77.1,7.044014,15.552209,24.261662,53.142117,5.1,4.6,113332.0,182.941086


In [439]:
# inspect for missing NaN values

df_.isna().sum()

State                                                                       10
County Name                                                                 10
e(0)                                                                         0
Percent of adults with less than a high school diploma, 2015-19             10
Percent of adults with a high school diploma only, 2015-19                  10
Percent of adults completing some college or associate's degree, 2015-19    10
Percent of adults with a bachelor's degree or higher, 2015-19               10
PCTPOV017_2019                                                              10
Unemployment_rate_2020                                                      10
Median_Household_Income_2019                                                10
Med_HH_Income_Percent_of_State_Total_2019                                   10
dtype: int64

In [440]:
# locate df_ rows with NaN values
# drop rows with NaN values

df_[df_.isna().any(axis=1)]

df_ = df_.dropna(axis = 0)

print(df_.shape)
df_.isna().sum()

(65652, 11)


State                                                                       0
County Name                                                                 0
e(0)                                                                        0
Percent of adults with less than a high school diploma, 2015-19             0
Percent of adults with a high school diploma only, 2015-19                  0
Percent of adults completing some college or associate's degree, 2015-19    0
Percent of adults with a bachelor's degree or higher, 2015-19               0
PCTPOV017_2019                                                              0
Unemployment_rate_2020                                                      0
Median_Household_Income_2019                                                0
Med_HH_Income_Percent_of_State_Total_2019                                   0
dtype: int64

In [441]:
df_ = df_.groupby(['State', 'County Name']).median().reset_index()

print("Number of Unique County Name values: {}".format(df_['County Name'].value_counts()))
df_.shape

Number of Unique County Name values: Washington    29
Jefferson     25
Franklin      24
Jackson       23
Lincoln       22
              ..
Conecuh        1
Estill         1
Georgetown     1
Buckingham     1
Cherry         1
Name: County Name, Length: 1781, dtype: int64


(3014, 11)

# Linear Regression Model Comparison Using GridSearchCV 

## Ordinary Least Squares Regression: No scaling

### Results:

OLS - No Scaling

Dataset shape: (3014, 1837) <br>
Training set score: 0.86 <br>
Test set score: 0.36

OLS - Standard Scaler applied

Scaled dataset shape: (3014, 1838) <br>
Training set score: 0.42 <br>
Test set score: -154413388028314289299838205952.00 <br>

In [442]:
df_.columns

Index(['State', 'County Name', 'e(0)',
       'Percent of adults with less than a high school diploma, 2015-19',
       'Percent of adults with a high school diploma only, 2015-19',
       'Percent of adults completing some college or associate's degree, 2015-19',
       'Percent of adults with a bachelor's degree or higher, 2015-19',
       'PCTPOV017_2019', 'Unemployment_rate_2020',
       'Median_Household_Income_2019',
       'Med_HH_Income_Percent_of_State_Total_2019'],
      dtype='object')

In [443]:
# onehotencode categorical variables

df_ = pd.get_dummies(data = df_, columns = ['State', 'County Name'], drop_first = True)

# train, test, split data

X_features = df_.drop('e(0)', axis = 1)
target = df_['e(0)']

X_train, X_test, y_train, y_test = train_test_split(X_features, target, train_size = 0.7)

print('After encoding categorical features the dataframe shape is: {}'.format(df_.shape))

After encoding categorical features the dataframe shape is: (3014, 1837)


In [429]:
y_train.shape, X_train.shape

((2109,), (2109, 1836))

In [444]:
lr = LinearRegression().fit(X_train, y_train)

print("Dataset shape: {}".format(df_.shape))
print("Training set score: {:.2f}".format(lr.score(X_train, y_train)))
print("Test set score: {:.2f}".format(lr.score(X_test, y_test)))

Dataset shape: (3014, 1837)
Training set score: 0.86
Test set score: 0.36


# Standard Scaling preprocessing before apply OLS

## Results: 

Dataset shape: (3014, 1838) <br>
Training set score: 0.87 <br>
Test set score: -213.76 <br>


Scaled dataset shape: (3014, 1838) <br>
Training set score: 0.42 <br>
Test set score: -154413388028314289299838205952.00 <br>

In [389]:
# Inspect dataframe columns with get.dummies() transformed

df_.columns

Index(['e(0)',
       'Percent of adults with less than a high school diploma, 2015-19',
       'Percent of adults with a high school diploma only, 2015-19',
       'Percent of adults completing some college or associate's degree, 2015-19',
       'Percent of adults with a bachelor's degree or higher, 2015-19',
       'PCTPOV017_2019', 'MEDHHINC_2019', 'Unemployment_rate_2020',
       'Median_Household_Income_2019',
       'Med_HH_Income_Percent_of_State_Total_2019',
       ...
       'County Name_Yoakum', 'County Name_Yolo', 'County Name_York',
       'County Name_Young', 'County Name_Yuba', 'County Name_Yukon Koyukuk',
       'County Name_Yuma', 'County Name_Zapata', 'County Name_Zavala',
       'County Name_Ziebach'],
      dtype='object', length=1838)

In [445]:
# StandardScaler() to df_
std_scaler = StandardScaler()
 
df_scaled = std_scaler.fit_transform(df_.to_numpy())
df_scaled = pd.DataFrame(df_scaled, columns=df_.columns)

print("Scaled dataset shape: {}".format(df_scaled.shape))

# train, test, split df_

X_features = df_scaled.drop('e(0)', axis = 1)
target = df_scaled['e(0)']

X_train, X_test, y_train, y_test = train_test_split(X_features, target, train_size = 0.7)

lr = LinearRegression().fit(X_train, y_train)

print("Training set score: {:.2f}".format(lr.score(X_train, y_train)))
print("Test set score: {:.2f}".format(lr.score(X_test, y_test)))

Scaled dataset shape: (3014, 1837)
Training set score: 0.18
Test set score: -163732003925225339730459099136.00


In [455]:
# plot predicted and actual values for test set.

y_pred = lr.predict(X_test)
df_Z = pd.DataFrame({'Income':X_test['Median_Household_Income_2019'], 'y test':y_test, 'y pred':y_pred})

df_Z

,Income,y test,y pred
1509,-1.014455,-0.435333,2.231445e-01
205,0.955114,0.963835,1.104980e+00
1239,0.027497,0.005501,5.043945e-01
377,-1.165144,-0.914500,-3.953049e+14
1038,0.896372,-0.377833,1.204609e+15
...,...,...,...
1895,0.870082,0.005501,6.274667e+14
1421,-0.945101,-1.642834,-6.862352e+14
382,0.912255,-0.991167,2.774628e+14
2007,0.890826,-0.205333,-2.202148e-01


In [456]:
#>>> from sklearn.metrics import r2_score
#>>> y_true = [3, -0.5, 2, 7]
#>>> y_pred = [2.5, 0.0, 2, 8]
#>>> r2_score(y_true, y_pred)

y_pred =lr.predict(X_train)

from sklearn.metrics import r2_score

r2_score(y_train, y_pred)

0.18492503675122274

In [457]:
df_Z.head()
fig = go.Figure()
fig = go.Figure(data=go.Scatter(x=df_Z["Income"],
                                y=df_Z['y test'],
                                mode='markers'))
                                
fig.add_trace(go.Scatter(x=df_Z["Income"],
                                y=df_Z['y pred'], mode = 'markers'))

fig.show()

# Lasso Regression using GridSearchCV

## Result:

### Lasso Regession - No Tuning

Training set score: 0.42 <br>
Test set score: 0.38 <br>
Training set r2_score: -0.1835086365759162 <br>
Test set r2_score: -0.1835086365759162

### Lasso Regression - Parameter Tuning

Test set score: 0.34 <br>
Best parameters: {'linear__alpha': 0.1, 'reducer__n_components': 7} <br>
Best cross_validation score: 0.39

In [458]:
df_

,e(0),"Percent of adults with less than a high school diploma, 2015-19","Percent of adults with a high school diploma only, 2015-19","Percent of adults completing some college or associate's degree, 2015-19","Percent of adults with a bachelor's degree or higher, 2015-19",PCTPOV017_2019,Unemployment_rate_2020,Median_Household_Income_2019,Med_HH_Income_Percent_of_State_Total_2019,State_Alaska,...,County Name_Yoakum,County Name_Yolo,County Name_York,County Name_Young,County Name_Yuba,County Name_Yukon Koyukuk,County Name_Yuma,County Name_Zapata,County Name_Zavala,County Name_Ziebach
0,74.7,11.483395,33.588459,28.356571,26.571573,15.9,4.9,58233.0,112.481888,0,...,0,0,0,0,0,0,0,0,0,0
1,78.2,9.193843,27.659616,31.284081,31.862459,13.5,5.6,59871.0,115.645828,0,...,0,0,0,0,0,0,0,0,0,0
2,73.5,26.786907,35.604542,26.029837,11.578713,41.0,7.0,35972.0,69.482918,0,...,0,0,0,0,0,0,0,0,0,0
3,74.1,20.942602,44.878773,23.800098,10.378526,25.9,6.6,47918.0,92.557610,0,...,0,0,0,0,0,0,0,0,0,0
4,76.9,19.509438,33.422131,33.975021,13.093413,21.0,4.1,52902.0,102.184624,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3009,78.8,7.213361,33.041271,37.246979,22.498390,10.1,7.4,80639.0,121.899567,0,...,0,0,0,0,0,0,0,0,0,0
3010,82.4,4.814409,14.876176,23.304277,57.005138,5.7,6.0,98837.0,149.408936,0,...,0,0,0,0,0,0,0,0,0,0
3011,77.4,7.258562,41.522678,35.189754,16.029003,9.6,6.3,70756.0,106.959732,0,...,0,0,0,0,0,0,0,0,0,0
3012,80.1,10.241615,29.751171,36.620987,23.386225,13.9,5.3,55122.0,83.326279,0,...,0,0,0,0,0,0,0,0,0,0


In [460]:
# lasso regressor, no parameter tuning, standardscaling

X_features = df_.drop('e(0)', axis = 1)
target = df_['e(0)']

X_train, X_test, y_train, y_test = train_test_split(X_features, target)

lasso = Lasso().fit(X_train, y_train)

y_train_pred =lr.predict(X_train)
y_test_pred =lr.predict(X_test)

print("Training set score: {:.2f}".format(lasso.score(X_train, y_train)))
print("Test set score: {:.2f}".format(lasso.score(X_test, y_test)))

print("Training set r2_score: {}".format(r2_score(y_train_pred, y_train)))
print("Test set r2_score: {}".format(r2_score(y_train_pred, y_train)))


Training set score: 0.40
Test set score: 0.41
Training set r2_score: -0.19239105411013746
Test set r2_score: -0.19239105411013746


In [462]:
# GridSearchCV Lasso Regressor using PCA() and Standard Scaler


# create steps

steps = [
    ('scaler', StandardScaler()),
    ('reducer', PCA()),
    ('linear', Lasso())]

#instantiate pipeline object

pipe = Pipeline(steps)

# train, test, split df_

X_features = df_.drop('e(0)', axis = 1)
target = df_['e(0)']

X_train, X_test, y_train, y_test = train_test_split(X_features, target)

# GridSearchCV parameters

param_grid = { 'reducer__n_components': [1, 2, 3, 4, 5, 6, 7],
              'linear__alpha':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]}


# split data into test train split

# instantiate GridSearchCV object using pipeline, parameters dict

grid_search = GridSearchCV(pipe, param_grid, cv=5, return_train_score = True)

grid_search.fit(X_train, y_train)


print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross_validation score: {:.2f}".format(grid_search.best_score_))


Test set score: 0.34
Best parameters: {'linear__alpha': 0.1, 'reducer__n_components': 7}
Best cross_validation score: 0.39


# Ridge Regression

## Results:

Test set score: 0.54 <br>
Best parameters: {'linear__alpha': 1.0} <br>
Best cross_validation score: 0.49

In [463]:
# create steps

steps = [
         ('linear', Ridge())]

#instantiate pipeline object

pipe = Pipeline(steps)

# train, test, split df_

X_features = df_.drop('e(0)', axis = 1)
target = df_['e(0)']

X_train, X_test, y_train, y_test = train_test_split(X_features, target)

# GridSearchCV parameters

param_grid = {
              'linear__alpha':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]}


# split data into test train split

# instantiate GridSearchCV object using pipeline, parameters dict

grid_search = GridSearchCV(pipe, param_grid, cv=5, return_train_score = True)

grid_search.fit(X_train, y_train)


print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross_validation score: {:.2f}".format(grid_search.best_score_))

Test set score: 0.54
Best parameters: {'linear__alpha': 1.0}
Best cross_validation score: 0.49


# Elastic Net

In [465]:
# create steps

steps = [
         ('linear', ElasticNet(max_iter=10000))]

#instantiate pipeline object

pipe = Pipeline(steps)

# train, test, split df_

X_features = df_.drop('e(0)', axis = 1)
target = df_['e(0)']

X_train, X_test, y_train, y_test = train_test_split(X_features, target)

# GridSearchCV parameters

param_grid = {'linear__l1_ratio':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],
              'linear__alpha':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]}


# split data into test train split

# instantiate GridSearchCV object using pipeline, parameters dict

grid_search = GridSearchCV(pipe, param_grid, cv=5, return_train_score = True)

grid_search.fit(X_train, y_train)


print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross_validation score: {:.2f}".format(grid_search.best_score_))

Test set score: 0.50
Best parameters: {'linear__alpha': 0.1, 'linear__l1_ratio': 0.1}
Best cross_validation score: 0.45


## Elastic net with StandardScaler(), Principal Component Analysis

## Results:

Test set score: 0.38 <br>
Best parameters: {'linear__alpha': 0.1, 'linear__l1_ratio': 0.4, 'reducer__n_components': 7} <br>
Best cross_validation score: 0.39


In [466]:
# Elastic net with StandardScaler(), Principal Component Analysis

# create steps

steps = [
    ('scaler', StandardScaler()),
    ('reducer', PCA()),
    ('linear', ElasticNet())]

# instantiate pipeline object

pipe = Pipeline(steps)

# train, test, split df_

X_features = df_.drop('e(0)', axis=1)
target = df_['e(0)']

X_train, X_test, y_train, y_test = train_test_split(X_features, target)

# GridSearchCV parameters

param_grid = {'reducer__n_components': [1, 2, 3, 4, 5, 6, 7],
              'linear__l1_ratio': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
              'linear__alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}


# split data into test train split

# instantiate GridSearchCV object using pipeline, parameters dict

grid_search = GridSearchCV(pipe, param_grid, cv=5, return_train_score=True)

grid_search.fit(X_train, y_train)


print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross_validation score: {:.2f}".format(grid_search.best_score_))

Test set score: 0.38
Best parameters: {'linear__alpha': 0.1, 'linear__l1_ratio': 0.4, 'reducer__n_components': 7}
Best cross_validation score: 0.39
